In [1]:
AGENT_NAME = '20 bin PPO 500 results\default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SAVE_DIR = r'20 bin PPO 500 results\binary classifier uACG results' + '/'
EPS = 0.03
ATK_NAME = 'bifurcated_uACG_DLloss_03_mask_time_solar_and_consumption'
CONSUMPTION_IDX = 26
SOLAR_IDX = 24

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.attacks.evasion import AutoConjugateGradient as ACG
#from KBMproject.mybb import BrendelBethgeAttack as BBA
from art.estimators.classification import PyTorchClassifier as classifier
from art.utils import to_categorical

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm

import KBMproject.utilities as utils

import json
from tqdm import tqdm

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
try: #try to load CityLearn schema
    schema = DataSet.get_schema(DATASET_NAME)
except: #load saved schema otherwise
    with open(DATASET_NAME, 'r') as file:
        schema = json.load(file)

Define RL agent

In [5]:
agent = PPO.load(path=f"{AGENT_NAME}",
                 print_system_info=True)
print('Model loaded from storage')

== CURRENT SYSTEM INFO ==
- OS: Windows-10-10.0.22631-SP0 10.0.22631
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0

== SAVED MODEL SYSTEM INFO ==
- OS: Windows-10-10.0.19045-SP0 10.0.19045
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.0
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0

Model loaded from storage


In [6]:
bins = agent.action_space[0].n
env = utils.make_discrete_env(schema=schema,  
                        action_bins=bins,
                        seed=42)

In [7]:
cols = env.observation_names

In [8]:
observation_masks = np.ones(agent.observation_space.shape)
observation_masks[0:6] = 0 #mask time features
observation_masks[SOLAR_IDX] = 0
observation_masks[CONSUMPTION_IDX] = 0

In [9]:
observation_masks

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1.])

In [9]:
class OutputCombinationWrapper(nn.Module):
    def __init__(self, base_model):
        super(OutputCombinationWrapper, self).__init__()
        self.base_model = base_model


    def forward(self, x):
        logits = self.base_model(x)
        lower_half, higher_half = torch.split(logits, logits.size(1) // 2, dim=1)
        
        # get the max of the lower and higher halves
        lower_max = torch.max(lower_half, dim=1)[0]
        higher_max = torch.max(higher_half, dim=1)[0]
        
        # concatenate the max of the lower and higher halves into a single tensor
        output = torch.cat((lower_max.unsqueeze(1), higher_max.unsqueeze(1)), dim=1)
        return output

In [10]:
agent_policy = OutputCombinationWrapper(utils.extract_actor(agent))

In [11]:
agent_classifier = classifier(
        model=agent_policy,
        loss=utils.CWLoss(), 
        nb_classes=2,
        input_shape=agent.observation_space.shape,
        device_type='gpu',
        clip_values = (agent.observation_space.low.min(),agent.observation_space.high.max()) #min and max values of each feature, brendle bethge attack only supports floats values and not array
        )

In [12]:
init = 50
iter = int(1000/init)
import logging
logging.basicConfig(level=logging.WARN)
kwargs = {"loss_type": None,  #attack is not targeted, but targets used in function below -> those are model predictions
          "eps": EPS, 
          "eps_step": 2*EPS, 
          "batch_size": 1, 
          "nb_random_init": init, #init, #try 0 to match myPGD
          "max_iter": iter, #iter, #try 100 to match myPGD
          "norm": "inf", 
          "verbose": False}
attack = ACG(estimator=agent_classifier, **kwargs,)

In [13]:
def eval_untargeted_attack(agent, env, atk, time_steps:int=None, mask:list=None):
    """Evaluates an SB3 agent subject to untargeted observation perturbations generated by an ART evasion attack"""
    obs_list = []
    adv_obs_list = []
    a_list = []
    adv_a_list = []
    asr = 0
    n_features = agent.observation_space.shape[0]

    observations = env.reset()
    if time_steps is None:
        time_steps = env.time_steps - 1
    if mask is None:
        mask=np.ones(n_features) #1 for all features

    pbar = tqdm(total=time_steps)
    for step in tqdm(range(time_steps)):

        obs_list.append(observations)
        actions = agent.predict(observations, deterministic=True)
        a_list.append(actions[0])
        if(actions[0] < agent.action_space[0].n//2): #has this caused an issue compared to myPGD? or does it just replace a prediction/forward pass? n/c from removing it
            target = to_categorical([0],2)
        else:
            target = to_categorical([1],2)

        adv_obs = np.expand_dims(observations, axis=0) #ART atks expect a 2d array
        adv_obs = atk.generate(adv_obs, 
                               y=target, 
                               mask=mask)
        adv_obs = np.squeeze(adv_obs) #CityLearn envs expect a 1d array
        
        a_adv, _ = agent.predict(adv_obs, deterministic=True)
        if a_adv[0]!=actions[0]: #check if an adversarial example was crafted
            asr+=1
            adv_obs_list.append(adv_obs)
        else:
            adv_obs_list.append(np.array([np.nan]*n_features)) #same shape as observations

        adv_a_list.append(a_adv[0])
        observations, _, _, _ = env.step(a_adv)

        #update progress bar including asr
        pbar.update(1)
        pbar.set_postfix({'ASR': asr/(step + 1)}, refresh=True)
        if env.done:
            break
    
    pbar.close()
    asr/=time_steps
    return utils.format_kpis(env), np.array(obs_list), np.array(adv_obs_list), np.array(a_list), np.array(adv_a_list), asr 

In [14]:
%%time
kpis, obs, adv_obs, actions, adv_actions, asr = eval_untargeted_attack(agent, 
                                                                        env, 
                                                                        attack,
                                                                        time_steps=None,
                                                                        mask=observation_masks,
                                                                        )

100%|██████████| 8759/8759 [6:58:06<00:00,  2.86s/it, ASR=0.223]


CPU times: total: 23min 32s
Wall time: 6h 58min 7s


prev ASR was 0.343

In [15]:
with open(SAVE_DIR+f'{ATK_NAME} parameters.json', 'w') as f:
    json.dump(kwargs, f)

In [16]:
display(kpis)

cost_function
annual_peak_average                      1.260144
carbon_emissions_total                   0.895974
cost_total                               0.824148
daily_one_minus_load_factor_average      0.983028
daily_peak_average                       0.953976
electricity_consumption_total            0.902861
monthly_one_minus_load_factor_average    0.988683
ramping_average                          1.224616
zero_net_energy                          1.112879
Name: District, dtype: float64

In [17]:
kpi = kpis

In [18]:
kpi_savename = SAVE_DIR+'KPIs.csv'
try:
    df_kpis = pd.read_csv(kpi_savename, index_col=0)
    df_kpis[ATK_NAME] = kpi.values
    df_kpis.to_csv(kpi_savename)
    print(f'{kpi_savename} updated')
except:
    kpi.name = ATK_NAME
    kpi.to_csv(kpi_savename)
    print(f'{kpi_savename} created')

20 bin PPO 500 results\binary classifier uACG results/KPIs.csv updated


In [19]:
df_obs = pd.DataFrame(obs)
df_obs.columns = cols
df_obs['a'] = actions
df_obs.to_csv(SAVE_DIR+ATK_NAME+' a-obs.csv')

In [20]:
df_obs = pd.DataFrame(adv_obs)
df_obs.columns = cols
df_obs['a'] = adv_actions
df_obs.to_csv(SAVE_DIR+ATK_NAME+' adv a-obs.csv')

In [21]:
asr_savename = SAVE_DIR+'ASRs.csv'
try:
    df_asrs = pd.read_csv(asr_savename)
    df_asrs[ATK_NAME] = asr
    df_asrs.to_csv(asr_savename)
    print(f'{asr_savename} updated')
except:
    asr = pd.Series([asr])
    asr.name = ATK_NAME
    asr.to_csv(asr_savename)
    print(f'{asr_savename} created')

20 bin PPO 500 results\binary classifier uACG results/ASRs.csv updated
